# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 2

## О документе

Версия 1.03

Модуль 2 для регионального этапа соревнований по кибериммунной автономности

## Подготовка к работе c модулем 2

Перенесите в блок ниже ваши реализации классов коммуникационного шлюза (CommunicationGateway), систему навигации (NavigationSystem), управления (ControlSystem)

In [50]:
from src.config import *
from src.communication_gateway import BaseCommunicationGateway
from src.event_types import Event
from multiprocessing import Queue


class CommunicationGateway(BaseCommunicationGateway):

    def _send_mission_to_consumers(self):
        """Метод для отправки данных в систему управления"""

        # Создание события для системы управление
        event_to_control = Event(
            self.event_source_name,
            CONTROL_SYSTEM_QUEUE_NAME,
            operation="set_mission",
            parameters=self._mission,
        )
        
        # Получение очереди системы управление
        control_q: Queue = self._queues_dir.get_queue(CONTROL_SYSTEM_QUEUE_NAME)
        
        # Отправка события в очередь
        control_q.put(event_to_control)
        
from src.control_system import BaseControlSystem


class ControlSystem(BaseControlSystem):
    
    def _send_speed_and_direction_to_consumers(self, speed: float, direction: float):
        """
        Метод для отправки значений скорости и направление в систему привода
        
        Args:
            speed (float): значение скорости
            direction (float): значение направления в градусах
            
        Returns:
            None: ничего не возвращает
        """
        
        # Создание события для отправки скорости 
        event_to_servos_speed = Event(
            self.event_source_name,
            SERVOS_QUEUE_NAME,
            operation='set_speed',
            parameters=speed
        )
        
        # Создание события для отправки напрваление 
        event_to_servos_direction = Event(
            self.event_source_name,
            SERVOS_QUEUE_NAME,
            operation='set_direction',
            parameters=direction
        )
        
        # Получение очереди привода управления
        servos_q: Queue = self._queues_dir.get_queue(SERVOS_QUEUE_NAME)
        
        # Отправка событий в очередь
        servos_q.put(event_to_servos_direction)
        servos_q.put(event_to_servos_speed)

    def _release_cargo(self):
        """Метод для отправки команды открытия грузового отсека"""

        event_to_cargo = Event(
            self.event_source_name,
            CARGO_BAY_QUEUE_NAME,
            operation='release_cargo',
            parameters=None
        )
        
        cargo_q: Queue = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        cargo_q.put(event_to_cargo)
        
    def _lock_cargo(self):
        """Метод для отправки команды закрытия грузового отсека"""
        
        event_to_cargo = Event(
            self.event_source_name,
            CARGO_BAY_QUEUE_NAME,
            operation='lock_cargo',
            parameters=None
        )
        
        # Получение очереди привода управления
        cargo_q: Queue = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        
        # Отправка событий в очередь
        cargo_q.put(event_to_cargo)
        
from src.navigation_system import BaseNavigationSystem

class NavigationSystem(BaseNavigationSystem):
     def _send_position_to_consumers(self):
        """Метод для отправки позиции в систему управления"""
        
        # Создание события для обновления позиции
        event_to_control = Event(
           self.event_source_name,
           CONTROL_SYSTEM_QUEUE_NAME,
           operation='position_update',
           parameters=self._position
        )
        
        # Получение очереди привода управления
        control_q: Queue = self._queues_dir.get_queue(CONTROL_SYSTEM_QUEUE_NAME)
        
        # Отправка событий в очередь
        control_q.put(event_to_control)
        


Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [51]:
afcs_present = True

В этом модуле изменим идентификатор машинки:

In [52]:
car_id = "m3" 

### Модуль 2. Отправка маршрутного здания и следование по маршруту

В рамках этого модуля отрабатывается только автономность. 
Никаких киберпрепятствий пока не будет. 

Что нужно сделать по пунктам:
1. Для заданных начальных и конечных точек создать маршрут, содержащий не менее пяти промежуточных путевых точек вручную или с использованием таких инструментов как APMPlanner, MissionPlanner. <br>Содержимое wpl файла нужно вставить в соответствующий кодовый блок блокнота, запустить симуляцию и убедиться, что машинка успешно проходит этот маршрут.
2. Задать скоростные ограничения в виде массива элементов GeoSpecificSpeedLimit
3. Запустить симуляцию


Формат WPL файла описан на этой странице: 

https://mavlink.io/en/file_formats/#mission_plain_text_file

Пример содержимого файла с текстовым описанием маршрута

In [53]:
wpl_file_content =  """QGC WPL 110
0	1	0	16	0	5	0	0	47.1426973397371682	39.7509250044822693	0	1
1	0	3	16	0	5	0	0	47.1426042967381775	39.7509115934371948	20	1
2	0	3	16	0	5	0	0	47.1424966585557854	39.7501927614212036	20	1
3	0	3	16	0	5	0	0	47.1424619953659132	39.7498708963394165	20	1
4	0	3	16	0	5	0	0	47.1424546978493879	39.7497072815895081	20	1
5	0	3	16	0	5	0	0	47.1425897017431694	39.7496724128723145	20	1
6	0	3	16	0	5	0	0	47.1429454536050514	39.7495248913764954	20	1
7	0	3	16	0	5	0	0	47.142963697226115	39.7497072815895081	20	1
8	0	3	16	0	5	0	0	47.1430074818910967	39.7499245405197144	20	1
9	0	3	16	0	5	0	0	47.1431461330923085	39.7499272227287292	20	1
"""


Для выполнения задания в блоке ниже  
1. замените имя файла с маршрутом (на свой), вот эту строку:
   ```python
   wpl_file = "module2.wpl"
   ```
2. измените список скоростных ограничений, вот этот массив:
   ```python
   speed_limits = [
    GeoSpecificSpeedLimit(...
    ```
   
Учтите, что вам нужно доставить груз как можно быстрее, но при этом не нарушить правила дорожного движения. 

Ограничения скорости
- населённый пункт - 60 км/ч
- пешеходная зона - 20 км/ч
- магистраль - 110 км/ч

Составьте маршрут доставки так, чтобы машинка доехала за минимальное время. Для определения оптимального маршрута допустимо пользоваться сторонними сервисами (например, Яндекс картами).

In [54]:
# создадим wpl файл и запишем туда маршрутное задание
wpl_file = "module2.wpl"

**Примечание**:
После создания собственного маршрута удалите перезапись содержимого, чтобы не потерять свои наработки!


In [55]:
with open(wpl_file, "w") as f:
    f.write(wpl_file_content)

**Напоминание**: не забудьте изменить скоростные ограничения (speed_limits) при задании маршрутного задания!

In [56]:
from src.wpl_parser import WPLParser

parser = WPLParser(wpl_file)    
points = parser.parse()
print(points)

from src.mission_type import GeoSpecificSpeedLimit
speed_limits = [
    GeoSpecificSpeedLimit(0, 30),
    GeoSpecificSpeedLimit(1, 60),
    GeoSpecificSpeedLimit(2, 50),
    GeoSpecificSpeedLimit(3, 20),
    GeoSpecificSpeedLimit(5, 60),
    GeoSpecificSpeedLimit(6, 20),
]

from src.mission_planner import Mission

home = points[0]
print(home)
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

[Point(47.14269733973717, 39.75092500448227, 0.0), Point(47.14260429673818, 39.750911593437195, 0.0), Point(47.142496658555785, 39.750192761421204, 0.0), Point(47.14246199536591, 39.74987089633942, 0.0), Point(47.14245469784939, 39.74970728158951, 0.0), Point(47.14258970174317, 39.749672412872314, 0.0), Point(47.14294545360505, 39.749524891376495, 0.0), Point(47.142963697226115, 39.74970728158951, 0.0), Point(47.1430074818911, 39.749924540519714, 0.0), Point(47.14314613309231, 39.74992722272873, 0.0)]
47 8m 33.7104s N, 39 45m 3.33002s E


Запускаем симуляцию с новым маршрутным заданием

In [57]:
from time import sleep
from geopy import Point as GeoPoint


from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer


# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 

if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)


communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

# у нас получилось довольно много блоков, используем класс SystemComponentsContainer
# для упрощения рутинной работы с ними - таким образом мы собираем все блоки машинки в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ])

system_components.start()

# ограничение поездки по времени
# параметр sleep - время в секундах,
# настройте этот параметр так, чтобы ваша машинка завершила маршрут
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(100)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][MISSION PLANNER] старт системы планирования заданий
[ИНФО][CARGO] старт блока грузового отсека
[ИНФО][CONTROL] старт системы управления
47 8m 33.7104s N, 39 45m 3.33002s E
47 8m 33.3755s N, 39 45m 3.28174s E
47 8m 32.988s N, 39 45m 0.693941s E
47 8m 32.8632s N, 39 44m 59.5352s E
47 8m 32.8369s N, 39 44m 58.9462s E
47 8m 33.3229s N, 39 44m 58.8207s E
47 8m 34.6036s N, 39 44m

На этом модуль 2 завершён, надеемся, у вас всё получилось! 
На что можно ещё обратить вниманием - можно ли как-то улучшить маршрут для ускорения доставки? (без нарушения ПДД) - например, использовать более скоростные трассы или срезать какие-то участки.

Если идей быстрых улучшений нет, то пришло время перейти к самому интересному модулю и познакомиться с киберпрепятствиями!

Для работы над модулем 3 задания откройте блокнот cyberimmunity--autonomous-car-m3.